In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import json
import csv
import pandas as pd
import urllib.request as urlReq
import wget


In [47]:
def searchSome(query,number=50):
    jsonFiles = []
    firstQuery = min(number,100)
    url = urlReq.urlopen("https://www.rijksmuseum.nl/api/en/collection/?ps="+str(firstQuery)+"&q=Q&key=t2KR0CWi&format=json".replace("Q", query))
    myCsv = jsonCSV(json.loads(url.read().decode()))
    if(number>100): 
        number = number - 100
        index = 2
        while number > 0 :
            firstQuery = min(number,100)
            url = urlReq.urlopen("https://www.rijksmuseum.nl/api/en/collection/?p="+str(index)+"&ps="
                                 +str(firstQuery)+"&q=Q&key=t2KR0CWi&format=json".replace("Q", query))
            jsonFiles.append(jsonCSV(json.loads(url.read().decode())))
            number = number - 100
            index +=1
        for csv in jsonFiles:
            myCsv = pd.concat((myCsv,csv),ignore_index = True)
            
    return myCsv
   
def jsonCSV(file):
    df = pd.DataFrame.from_dict(file['artObjects'])
    return df
def downloadImages(csv, directory):
    for i in range(csv.shape[0]):
        if(csv['webImage'][i] is not None):
            wget.download(csv['webImage'][i]['url'],out=directory+"/image"+str(i)+".jpg")
    

In [48]:
mCsv= searchSome('rembrandt',250)


In [49]:
downloadImages(mCsv,"../rembrandt")


100% [..........................................................] 57939 / 57939